In [56]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [57]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [58]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Józef,Mąkulski,77040400059,Wrocław,ul. Ludwika Zamenhofa 62/31,48955127740,jozef.makulski@kuba.pl,2014-01-01,None,menadżer,74.8
1,2,Henryka,Chojecka,83071992704,Wrocław,ul. 56 Pułku Piechoty 77/27,48561732567,henryka.chojecka@kuba.pl,2014-01-01,None,sprzedawca,49.2
2,3,Karol,Dyda,75020644237,Wrocław,ul. 1 Maja 66/17,48699374408,karol.dyda@kuba.pl,2014-01-01,None,sprzedawca,53.8
3,4,Beata,Borysiuk,69121565764,Wrocław,ul. Azaliowa 2/31,48158016406,beata.borysiuk@kuba.pl,2014-01-01,2017-05-14,mechanik,48.5
12,5,Krystian,Marchwiak,79061647194,Gdynia,ul. inż. A. Hoffmanna 85,48336806883,krystian.marchwiak@kuba.pl,2014-01-01,None,informatyk,51.6
4,6,Janusz,Kuźniacki,75062829492,Będzin,ul. Nad Koleją 66/38,48384791976,janusz.kuzniacki@kuba.pl,2015-08-16,None,mechanik,47.7
5,7,Urszula,Cichoń,76072380524,Jelenia Góra,ul. Modrzewiowa 14/23,48280695083,urszula.cichon@kuba.pl,2015-09-06,None,mechanik,57.2
6,8,Dawid,Marszałek,75012465015,Poznań,ul. Ustrońska 9/19,48679387925,dawid.marszalek@kuba.pl,2015-10-13,2016-04-12,mechanik,51.8
7,9,Grażyna,Zawadzka,64121561123,Wrocław,ul. 29 Listopada 12/50,48947590290,grazyna.zawadzka@kuba.pl,2016-05-19,None,pomocnik mechanika,42.8
8,10,Radosław,Braun,89122195915,Wrocław,ul. Zielna 59/37,48813333463,radoslaw.braun@kuba.pl,2016-12-23,2017-04-29,pomocnik mechanika,40.0


In [59]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [60]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [61]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [62]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [63]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [64]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [65]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [66]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [67]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [68]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Wanda,Kluczkowska,K,70112991976,Wrocław,ul. Igielna 41,False,48357063469,w_kluczkowska5384@gmail.com
1,2,Dorota,Szembek,K,99020880037,Poznań,ul. Wierzbowa 42/44,True,48966357647,dszembek99@gmail.com
2,3,Halina,Baranowska,K,06241975659,Zakopane,ul. Załawie 9,False,48061532430,n065ohif506@opayq.com
3,4,Ewa,Najda,K,98081610658,Wrocław,ul. Jaśminowa 80/42,False,48741416864,e_na@gmail.com
4,5,Waldemar,Lecko,M,78101392698,Wrocław,ul. Źródlana 22/35,False,48044055932,lecko19787@onet.pl
...,...,...,...,...,...,...,...,...,...,...
1508,1509,Monika,Pabich,K,05321026359,Wrocław,ul. Szewska 55/14,False,48497798466,pabic2005@gmail.com
1509,1510,Orlan,Mizera,M,03260857315,Kraków,ul. Bolesława Limanowskiego 32,False,48741052421,snqwwm54a@opayq.com
1510,1511,Katarzyna,Dutka,K,70110653670,Płońsk,ul. Przemysłowa 91,False,48696989015,k-dutk70@onet.pl
1511,1512,Amelia,Balicka,K,93011175319,Wrocław,ul. Darżlubska 35/10,False,48490456399,ba.93@vp.pl


In [69]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Wanda,Kluczkowska,K,70112991976,Wrocław,ul. Igielna 41,False,48357063469,w_kluczkowska5384@gmail.com
1,2,Dorota,Szembek,K,99020880037,Poznań,ul. Wierzbowa 42/44,True,48966357647,dszembek99@gmail.com
2,3,Halina,Baranowska,K,06241975659,Zakopane,ul. Załawie 9,False,48061532430,n065ohif506@opayq.com
3,4,Ewa,Najda,K,98081610658,Wrocław,ul. Jaśminowa 80/42,False,48741416864,e_na@gmail.com
4,5,Waldemar,Lecko,M,78101392698,Wrocław,ul. Źródlana 22/35,False,48044055932,lecko19787@onet.pl
...,...,...,...,...,...,...,...,...,...,...
1508,1509,Monika,Pabich,K,05321026359,Wrocław,ul. Szewska 55/14,False,48497798466,pabic2005@gmail.com
1509,1510,Orlan,Mizera,M,03260857315,Kraków,ul. Bolesława Limanowskiego 32,False,48741052421,snqwwm54a@opayq.com
1510,1511,Katarzyna,Dutka,K,70110653670,Płońsk,ul. Przemysłowa 91,False,48696989015,k-dutk70@onet.pl
1511,1512,Amelia,Balicka,K,93011175319,Wrocław,ul. Darżlubska 35/10,False,48490456399,ba.93@vp.pl


In [70]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [71]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(67)

In [72]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(68)

# auta

In [73]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [74]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [75]:
df_samochody = create_car_table(max(car_ids))

In [76]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Lexus,NX 300,4,2006,Sedan,False,srebrny,Elektryczny/Benzyna,3.2,197,Automatyczna,5,5,False
1,2,BMW,330,4,2008,Kombi,True,szary,Diesel,3.9,252,Automatyczna,3,2,False
2,3,Alfa,Romeo Giulietta,4,2015,Kombi,False,biały,Diesel,1.9,120,Manualna,5,5,False
3,4,Audi,A4,4,2005,Kombi,True,czarny,Benzyna,2.6,170,Automatyczna,3,5,False
4,5,Mercedes-Benz,C 180,4,2012,Hatchback,False,szary,Benzyna,2.3,156,Automatyczna,3,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,976,Opel,Astra,4,2004,Hatchback,False,czarny,Diesel,2.0,140,Manualna,5,2,False
976,977,Ford,Mondeo,4,2002,Sedan,False,biały,Diesel,2.9,188,Automatyczna,5,5,True
977,978,Ford,Tourneo Connect,4,2014,SUV,False,srebrny,Benzyna,1.6,101,Manualna,5,2,False
978,979,Renault,Laguna,4,2004,SUV,True,szary,Diesel,1.8,110,Manualna,5,5,True


In [77]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [78]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna', 'Półautomatyczna'], dtype=object)

# Usługi

In [79]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [80]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [81]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [82]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_20636\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [83]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [84]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,3,2014-01-02,66700.0
1,2,2,2,3,2014-01-02,76300.0
2,3,3,3,3,2014-01-03,35500.0
3,4,4,4,2,2014-01-09,46500.0
4,5,5,5,2,2014-01-10,98300.0
...,...,...,...,...,...,...
938,939,976,902,2,2020-12-20,34700.0
939,940,977,1510,2,2020-12-23,40100.0
940,941,820,1340,3,2020-12-27,36800.0
941,942,979,1512,3,2020-12-29,20400.0


In [85]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [86]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [109]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():
        sell_prices[i] *= 0.97

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_20636\1097819302.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():


[np.int64(1), np.int64(18), np.int64(1), np.int64(6), np.int64(18), np.int64(6), np.int64(1), np.int64(1), np.int64(12), np.int64(6), np.int64(12), np.int64(1), np.int64(6), np.int64(6), np.int64(1), np.int64(6), np.int64(1), np.int64(18), np.int64(1), np.int64(18), np.int64(12), np.int64(18), np.int64(1), np.int64(18), np.int64(6), np.int64(1), np.int64(6), np.int64(1), np.int64(1), np.int64(1), np.int64(6), np.int64(6), np.int64(6), np.int64(12), np.int64(1), np.int64(12), np.int64(12), np.int64(12), np.int64(6), np.int64(1), np.int64(6), np.int64(12), np.int64(1), np.int64(1), np.int64(1), np.int64(18), np.int64(12), np.int64(12), np.int64(12), np.int64(1), np.int64(6), np.int64(6), np.int64(12), np.int64(6), np.int64(12), np.int64(1), np.int64(1), np.int64(12), np.int64(1), np.int64(1), np.int64(6), np.int64(6), np.int64(1), np.int64(6), np.int64(18), np.int64(6), np.int64(18), np.int64(6), np.int64(1), np.int64(18), np.int64(1), np.int64(6), np.int64(1), np.int64(1), np.int64(1), 

In [110]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,1,21,3,86400.00,1,2014-01-31
1,2,17,22,3,37312.00,18,2014-02-04
2,3,3,23,2,48300.00,1,2014-02-06
3,4,14,24,3,14994.00,6,2014-02-06
4,5,9,25,3,149191.82,18,2014-02-09
...,...,...,...,...,...,...,...
932,933,974,521,3,36108.00,6,2020-12-17
933,934,975,1507,2,42534.00,6,2020-12-20
934,935,88,1508,2,23256.00,6,2020-12-20
935,936,971,1509,2,43962.00,6,2020-12-20


In [89]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [90]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [91]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(29), array([10, 62])), (np.int64(32), array([44])), (np.int64(40), array([12])), (np.int64(45), array([9])), (np.int64(50), array([ 0, 32])), (np.int64(51), array([17])), (np.int64(61), array([61, 57])), (np.int64(65), array([61])), (np.int64(78), array([9])), (np.int64(87), array([10])), (np.int64(92), array([53, 46])), (np.int64(94), array([32,  3])), (np.int64(97), array([13])), (np.int64(106), array([42])), (np.int64(109), array([13, 58, 57])), (np.int64(112), array([47])), (np.int64(147), array([33, 12])), (np.int64(158), array([36, 62])), (np.int64(159), array([46])), (np.int64(167), array([59])), (np.int64(170), array([39])), (np.int64(177), array([17, 59])), (np.int64(178), array([15, 38])), (np.int64(187), array([7])), (np.int64(189), array([0])), (np.int64(193), array([55])), (np.int64(196), array([28, 29])), (np.int64(202), array([15])), (np.int64(204), array([11])), (np.int64(211), array([55,  5])), (np.int64(214), array([ 8, 21])), (np.int64(217), array([37])), 

In [92]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [93]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Drzwi lewe przednie,Opel Corsa srebrny,361,1,1
1,Lusterko zewnętrzne prawe,Opel Corsa srebrny,22,1,1
2,Kolektor wydechowy,,500,2,1
3,Drzwi lewe tylne,Volkswagen up! biały,281,3,1
4,Drzwi prawe przednie,Volkswagen Golf Sportsvan srebrny,550,4,1
...,...,...,...,...,...
164,Tłumik,,200,117,1
165,Światło mgłowe,Audi Q2,57,118,1
166,Amortyzator,,150,119,4
167,Drzwi prawe przednie,Ford Tourneo Connect srebrny,499,120,1


# Wyposażenie

In [94]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
#df_equipment['cena_części'] = equipment_price.astype(float)

In [95]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Drzwi lewe przednie,Opel Corsa srebrny,361.0
1,2,Lusterko zewnętrzne prawe,Opel Corsa srebrny,22.0
2,3,Kolektor wydechowy,,500.0
3,4,Drzwi lewe tylne,Volkswagen up! biały,281.0
4,5,Drzwi prawe przednie,Volkswagen Golf Sportsvan srebrny,550.0
...,...,...,...,...
161,133,Kratka,Skoda Rapid/Spaceback czarny,304.0
162,134,Maska,Skoda Octavia szary,730.0
163,135,Klocek hamulcowy,Audi Q3,27.0
165,136,Światło mgłowe,Audi Q2,57.0


In [96]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi - uzupełnienie

In [97]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [98]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,29,56,8,2014-03-21,500
1,2,32,61,4,2014-04-01,650
2,3,40,75,7,2014-04-22,360
3,4,45,85,7,2014-04-30,710
4,5,50,93,6,2014-05-08,630
...,...,...,...,...,...,...
115,116,528,101,6,2020-09-06,140
116,117,955,1482,4,2020-10-11,260
117,118,970,1499,8,2020-11-26,70
118,119,547,926,6,2020-11-28,780


In [99]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [100]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [101]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,1,2,1
2,3,2,3,1
3,4,3,4,1
4,5,4,5,1
...,...,...,...,...
164,165,117,15,1
165,166,118,136,1
166,167,119,116,4
167,168,120,137,1


In [102]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [103]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


In [104]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,665.0,2.0
1,2.0,2.0,110.0,6.0
2,3.0,3.0,4420.0,11.0
3,4.0,4.0,493.0,2.0
4,5.0,5.0,520.0,1.0
...,...,...,...,...
132,133.0,133.0,1442.0,6.0
133,134.0,134.0,664.0,1.0
134,135.0,135.0,148.0,7.0
135,136.0,136.0,259.0,5.0
